In [1]:
import re
import os
import numpy as np
import Preprocess as utils
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential, Input
from keras.layers import GlobalMaxPooling1D, Dropout, Dense, LSTM

/home/arko/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Data and Splitting into train and test set

In [2]:
data = utils.load('SMS_Spam_v2.csv')
sms_data, len_sms = utils.make_word_container(data)
print('\nMaximum Sentence Length in entire dataset = {}'.format(len_sms))
padded_container = utils.No_Word_padding(sms_data, len_sms)


Maximum Sentence Length in entire dataset = 191


In [3]:
#Gensim's Word2Vec
vector_size = 100
word2vec,_ = utils.Word2Vec(padded_container, vector_size, 1)

Word2Vec Model details = Word2Vec(vocab=9539, size=100, alpha=0.025)


In [4]:
sms_embedded = utils.Word_Embedding(word2vec, vector_size, padded_container)
print('The SMS Embedded tensor shape = {}'.format(sms_embedded.shape))

The SMS Embedded tensor shape = (11133, 191, 100)


In [5]:
labels = utils.create_labels(data)
train_x, test_x, train_y, test_y = train_test_split(sms_embedded, labels, test_size = 0.10, random_state = 42)
print('Training Tweets embedded tensor shape = {}'.format(train_x.shape))
print('Training Sentiment labels shape = {}'.format(train_y.shape))
print('Testing Tweets embedded tensor shape = {}'.format(test_x.shape))
print('Testing Sentiment labels tensor shape = {}'.format(test_y.shape))

Training Tweets embedded tensor shape = (10019, 191, 100)
Training Sentiment labels shape = (10019, 1)
Testing Tweets embedded tensor shape = (1114, 191, 100)
Testing Sentiment labels tensor shape = (1114, 1)


# Architecture

In [6]:
input_sms = Input(shape = (len_sms,vector_size,))

def Network(input_data):
    x = LSTM(units = 50, return_sequences = True, kernel_initializer = keras.initializers.lecun_normal(seed = None), unit_forget_bias = True)(input_data)
    x = GlobalMaxPooling1D()(x)
    z = Dense(units = 100, activation = 'relu')(x)
    z = Dropout(0.2)(z)
    z = Dense(units = 50, activation = 'relu')(x)
    z = Dropout(0.2)(z)
    z = Dense(units = 20, activation = 'relu')(z)
    z = Dropout(0.2)(z)
    z = Dense(units = 10, activation = 'relu')(z)
    z = Dropout(0.2)(z)
    output = Dense(units = 1, activation = 'sigmoid')(z)
    model = Model(inputs = input_data, outputs = output)
    model.summary()
    return model 

def compile_and_train(model, num_epochs): 
    model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['acc']) 
    history = model.fit([train_x], train_y, batch_size=1, epochs=num_epochs)
    return history

# Training

In [7]:
#Instantiation and Training
model = Network(input_sms)
history = compile_and_train(model, 2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 191, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 191, 50)           30200     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0         
__________

# Testing on Unseen Test Data

In [9]:
loss,acc = model.evaluate(test_x,test_y)
print('Test Loss = {}'.format(loss))
print('Test Accuracy = {}'.format(acc))

1114/1114 [==============================] - 1s 1ms/step
Test Loss = 0.0829325877096591
Test Accuracy = 0.9775583482944344
